<a href="https://colab.research.google.com/github/NNLogic/NNMPPublic/blob/main/%5BNNMP%5DNNSlide_Data_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#説明 / Description
PDF/PNG/JPGからNNMP用のNNSlideデータを作成するスクリプトになります。

　-------------------------------------------------------

　Betaテスト版です。

　★変換設定を実行後、変換してください。

　※ファイルサイズが大きい場合、Googleドライブから変換することを推奨します。

　-------------------------------------------------------

　This is a beta test version.

　★Please execute the steps in the following order: Settings → Convert.

　※If the file size is large, it is recommended to convert it from Google Drive.
The script to create a NNSLide image file for NNMP from PDF/PNG/JPG:

　-------------------------------------------------------


In [ ]:
from pkgutil import extend_path
from google.colab import files
import os
import shutil

import ipywidgets as widgets
from google.colab import drive


# @title #変換設定 / Settings
#@markdown ↓ファイル名を表示する場合、未入力にしてください。/  If you want to display the file name, please leave it blank.
rename = "" #@param {type:"string"}
#@markdown ↓ファイル名の末尾番号を画像位置にする場合、"off"にしてください。/  If you want to use the file name's trailing number for the image position, set "off".
renumber = "on" #@param ["on", "off"]

max_size = '4096' #@param ["1024", "2048", "4096"]
#max_size = '8192' #@param ["1024", "2048", "4096", "8192", "Unlimited"]

load_type = "PNG, JPG" #@param ["PDF", "PNG, JPG"]

load_from = "Google Drive" #@param ["Google Drive", "Local Computer"]
save_to = "Local Computer" #@param ["Google Drive", "Local Computer"]

win_format = "DXT1Crunched" #@param ["DXT1","DXT1Crunched"]
#win_format = "DXT1" #@param ["DXT1","DXT5","DXT1Crunched","DXT5Crunched"]

android_format = "ASTC_8x8"  #@param ["ASTC_4x4","ASTC_5x5","ASTC_6x6","ASTC_8x8","ASTC_10x10","ASTC_12x12"]

#@markdown ↓Google Driveの場合、指定してください。/ In the case of Google Drive, please specify.
#@markdown 　PDFはファイルパス、PNG/JPGはフォルダパス / For PDF files, provide the file path, and for PNG/JPG files, provide the folder path.
drive_path = "/content/drive/MyDrive/sample3" #@param {type:"string"}


if (load_from == "Google Drive") or (save_to == "Google Drive"):
  if not os.path.exists(drive_path):
    if 'drive' in locals():
      drive.flush_and_unmount()
      drive.mount('/content/drive')


#@title [1]　ファイルを読込 / Loading files
%cd  /content

if 'rename' not in locals():
    rename = ""
if 'max_width' not in locals():
    max_width = 1024



!rm -f *.pdf *.png *.jpg  *.dds *.crn *.json

def get_last_folder_path(path):
    path = path.rstrip(os.path.sep)
    dir_path, folder_name = os.path.split(path)
    return os.path.basename(folder_name)

if load_type ==  "PNG, JPG":
  image_files = []
  if load_from == "Local Computer":
    upload_files = files.upload()
    image_files = [file for file in upload_files if file.endswith('.png') or file.endswith('.jpg')]

  if load_from == "Google Drive":
    if not os.path.exists(drive_path):
      if 'drive' in locals():
        drive.flush_and_unmount()
        drive.mount('/content/drive')

    copy_dest = "/content"
    os.chdir(drive_path)
    output_file_name =  get_last_folder_path(drive_path)

    image_files = [file for file in os.listdir() if file.endswith('.png') or file.endswith('.jpg')]
    no=0
    for file in image_files:
      no = no + 1
      src_path = os.path.join(drive_path, file)
      dest_path = os.path.join(copy_dest, file)
      shutil.copy(src_path, dest_path)
    os.chdir("/content")
    image_files = [file for file in os.listdir() if file.endswith('.png') or file.endswith('.jpg')]

  no=0
  for file in image_files:
    no = no + 1
    if rename != "":
      file_name, file_extension = os.path.splitext(file)
      new_name =  rename+"-"+str(no)+file_extension
      os.rename(file, os.path.join(os.path.dirname(file), new_name))

if load_type ==  "PDF":
  pdf_file_name =""
  base_file_name =""

  if load_from == "Local Computer":
    result = files.upload()
    pdf_file_name = list(result.keys())[0]
    base_file_name = os.path.splitext(pdf_file_name)[0]
    output_file_name = base_file_name

  if load_from == "Google Drive":
    if not os.path.exists(drive_path):
      print("Error:drive_path is invalid")
      raise SystemExit
    copy_dest = "/content"  # コピー先のディレクトリパスを指定してください
    pdf_file_name = os.path.basename(drive_path)
    base_file_name = os.path.splitext(pdf_file_name)[0]
    output_file_name = base_file_name
    dest_path = os.path.join(copy_dest, pdf_file_name)
    shutil.copy(drive_path, dest_path)

  !apt-get install -y poppler-utils

  output_name = base_file_name if rename == "" else rename;

  if max_width == 'Unlimited':
    !pdftoppm -png "$pdf_file_name" "$output_name"
  else:
    !pdftoppm -scale-to $max_width -png "$pdf_file_name"  "$output_name"



#変換 / Convert


In [ ]:
#@title [1]NNSlideデータへの変換 / Converting to NNSlide Data

!sudo apt-get install imagemagick
!pip install wand
!sudo sed -i 's/<policy domain="resource" name="memory" value="256MiB"\/>/<policy domain="resource" name="memory" value="10GiB"\/>/' /etc/ImageMagick-6/policy.xml

!pip install pillow


import os, sys
import datetime
import base64
import json
import re
import subprocess
import tarfile
from PIL.ExifTags import TAGS
from PIL import Image as PilImage
from wand.image import Image as WandImage

%cd  /content

image_type_enc_dict = {"ASTC_4x4":"4x4",
              "ASTC_5x5":"5x5",
              "ASTC_6x6":"6x6",
              "ASTC_8x8":"8x8",
              "ASTC_10x10":"10x10",
              "ASTC_12x12":"12x12",
              };

image_type_dict = {"ASTC_4x4":48,
              "ASTC_5x5":49,
              "ASTC_6x6":50,
              "ASTC_8x8":51,
              "ASTC_10x10":52,
              "ASTC_12x12":53,
              "DXT1":10,
              "DXT5":10,
              "DXT1Crunched":28,
              "DXT5Crunched":29,
              };

#ASTC
filename="astcenc-4.7.0-linux-x64.zip"
if not os.path.isfile(filename):
    !wget https://github.com/ARM-software/astc-encoder/releases/download/4.7.0/astcenc-4.7.0-linux-x64.zip
    !mkdir -p astc
    !unzip -o astcenc-4.7.0-linux-x64.zip -d astc

def encode_astc(input_image, output_image, blocksize="8x8", quality="-medium"):
    command = [
        "astc/bin/astcenc-avx2", "-cl", input_image, output_image,
        blocksize, quality
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode == 0:
        print("ASTC",input_image,"successful!")
    else:
        print("ASTC",input_image,"failed:", result.stderr)

#ASTC


#DXT
#https://github.com/Unity-Technologies/crunch/tree/unity Original
#https://github.com/DaemonEngine/crunch latest info

!git clone https://github.com/NNLogic/crunch.git dxt
!chmod 777 dxt/crunch

image_type_enc_dxt_dict = {"DXT1":"-fileformat dds -dxt1 -quality 255 -bitrate 1.5 ",
              "DXT5":"-fileformat dds -dxt5 -quality 255 -bitrate 1.5 ",
              "DXT1Crunched":"-fileformat crn -dxt1 -quality 255 -bitrate 4.0 ",
              "DXT5Crunched":"-fileformat crn -dxt5 -quality 255 -bitrate 4.0 ",
              };

def encode_dxt(input_image,enc_type):
    command = [
        "dxt/crunch", "-file",input_image,image_type_enc_dxt_dict[enc_type],
        "-mipmode none -helperThreads 4  -imagestats -out dxt/output.dds"
    ]
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode == 0:
        print("DXT",input_image," successful!")
    else:
        print("DXT",input_image,"failed:", result.stderr)
#DXT


def extract_number(file_name):
    match = re.search(r"\d+$", os.path.splitext(file_name)[0])
    return int(match.group()) if match else 0

def change_extension(file_name, new_extension):
    base_name = os.path.splitext(file_name)[0]  # 拡張子を除いたファイル名を取得
    new_file_name = f"{base_name}.{new_extension}"  # 新しいファイル名を組み立てる
    return new_file_name

def extract_exposure_time(exif_info):
    exposure_time = exif_info.get("ExposureTime", "Not Available")

    if exposure_time != "Not Available":
        return str(exposure_time.real)
    else:
        return ""


if 'max_width' not in locals():
    max_width = "1024"
if 'rename' not in locals():
    rename = ""
if 'renumber' not in locals():
    renumber = "on"

no = 0
images = []
byte_data = []
files = os.listdir("./")
sorted_files = sorted(files, key=extract_number)

for file_name in sorted_files:

    if file_name.endswith(".png") or file_name.endswith(".jpg"):

      #EXIF
      im = PilImage.open(file_name)
      exif_info = None
      exif_data = im._getexif()
      if exif_data:
          exif_info = {}
          # タグとその値のペアを辞書に格納
          for tag, value in exif_data.items():
              tag_name = TAGS.get(tag, tag)
              exif_info[tag_name] = value

      #IMAGE
      im = WandImage(filename=file_name)

      target_width = im.width
      target_height = im.height

      if max_size != "Unlimited":
        width_ratio = int(max_size) / im.width
        height_ratio = int(max_size)  / im.height
        ratio = min(width_ratio, height_ratio)

        # 新しいサイズを計算
        target_width = int(im.width * ratio)
        target_height = int(im.height * ratio)

      new_width = 4 * (target_width // 4)
      new_height = 4 * (target_height // 4)

      if new_width != target_width or new_height != target_height:
        im.resize(new_width, new_height)

      im.flip()

      im.format = 'png'
      im.save(filename='astc/output.png')
      shutil.copy("/content/astc/output.png", "/content/dxt/output.png")

      #ASTC
      encode_astc("astc/output.png", "astc/output.astc",image_type_enc_dict[android_format])
      type_astc = image_type_dict[android_format]

      with open("astc/output.astc", 'rb') as file:
        file.seek(16)
        astc_data = file.read()

      #DXT
      encode_dxt("dxt/output.png",win_format)
      im_type = image_type_dict[win_format]

      with open("dxt/output.dds", 'rb') as file:
        dxt_data = file.read()

      if win_format == "DXT1" or win_format == "DXT5" :
        dxt_data = dxt_data[128:]

      #COMMON
      if renumber == "off":
        match = re.search(r"\d+$", os.path.splitext(file_name)[0])
        no = int(match.group()) if match else no + 1
        no = 200 if no > 200 else 1 if no < 1 else no
      else:
         no = no + 1

      nnslide_tex_dict = {"name":os.path.splitext(file_name)[0]};

      if exif_info:
          exif_dict = {
            "FNumber":str(exif_info.get("FNumber","")),
            "ShutterSpeed":extract_exposure_time(exif_info),
            "ISOSpeedRatings":str(exif_info.get("ISOSpeedRatings","")),
            "WhiteBalance": "Auto" if exif_info.get("WhiteBalance", 1) == 0 else "Manual",

            "FocalLength":str(exif_info.get("FocalLength","")),
            "FocalLengthIn35mmFilm":str(exif_info.get("FocalLengthIn35mmFilm","")),

            "Make":str(exif_info.get("Make","")),
            "Model":str(exif_info.get("Model","")),
            "LensMake":str(exif_info.get("LensMake","")),
            "LensModel":str(exif_info.get("LensModel",""))
          };
          nnslide_tex_dict["exif"] = exif_dict;


      byte_info  = json.dumps(nnslide_tex_dict).encode('utf-8')

      #DXT
      byte_image = bytearray(40)
      byte_image += byte_info
      byte_image += dxt_data

      byte_image[0:4] = len(byte_image).to_bytes(4, byteorder='little')
      byte_image[4:8] =no.to_bytes(4, byteorder='little')
      byte_image[8:12] = im_type.to_bytes(4, byteorder='little')
      byte_image[24:28] =  im.width.to_bytes(4, byteorder='little')
      byte_image[28:32] =  im.height.to_bytes(4, byteorder='little')
      byte_image[32:36] = len(byte_info).to_bytes(4, byteorder='little')
      byte_image[36:40] = len(dxt_data).to_bytes(4, byteorder='little')

      #ASTC
      byte_image_astc = bytearray(40)
      byte_image_astc += byte_info
      byte_image_astc += astc_data

      byte_image_astc[0:4] = len(byte_image_astc).to_bytes(4, byteorder='little')
      byte_image_astc[4:8] = no.to_bytes(4, byteorder='little')
      byte_image_astc[8:12] = type_astc.to_bytes(4, byteorder='little')
      byte_image_astc[24:28] =  im.width.to_bytes(4, byteorder='little')
      byte_image_astc[28:32] =  im.height.to_bytes(4, byteorder='little')
      byte_image_astc[32:36] = len(byte_info).to_bytes(4, byteorder='little')
      byte_image_astc[36:40] = len(astc_data).to_bytes(4, byteorder='little')

      byte_data.append(byte_image)
      byte_data.append(byte_image_astc)


##############################################
dict= {"version":"001","images":images}
##############################################

version  = 1;
options  = 0;

current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime("%Y%m%d_%H%M%S")

if 'output_file_name' not in locals():
    output_file_name  = formatted_datetime

nnslide_dict = {"name":output_file_name};

byte_format = bytearray(20)
byte_info  = json.dumps(nnslide_dict).encode('utf-8')

byte_format +=byte_info;

byte_format[0:4] = len(byte_format).to_bytes(4, byteorder='little')
byte_format[4:8] = version.to_bytes(4, byteorder='little')
byte_format[8:12] = options.to_bytes(4, byteorder='little')
byte_format[12:16] = int(len(byte_data)/2).to_bytes(4, byteorder='little')
byte_format[16:20] = len(byte_info).to_bytes(4, byteorder='little')

byte_format+= b''.join(byte_data)

byte_file_h = bytearray(14)
byte_file_h[0] = b'N'[0]
byte_file_h[1] = b'S'[0]

byte_file_h+=byte_format
byte_file_h[2:6] = len(byte_file_h).to_bytes(4, byteorder='little')
byte_data = byte_file_h
##############################################


# output
nnslide_file_name = output_file_name + '.bin'
with open(nnslide_file_name, "wb") as outputFile:
  outputFile.write(byte_data)

In [ ]:
#@title [2]ファイルのダウンロード / Downloading the file

from google.colab import files
import shutil

def create_directory_path(path):
    if os.path.isfile(path):
        path = os.path.dirname(path)
    if not path.endswith(os.path.sep):
        path += os.path.sep
    return path

if save_to == "Local Computer":
  files.download(nnslide_file_name)

if save_to == "Google Drive":
  if not os.path.exists(drive_path):
    if 'drive' in locals():
      drive.flush_and_unmount()
      drive.mount('/content/drive')
  shutil.copyfile(nnslide_file_name, create_directory_path(drive_path)+"/"+nnslide_file_name)


#その他 / Misc.


In [ ]:
#@title [Ex1]GitHub　URL変換 / Make GitHub URL
#@markdown GitHubのURLからNNPosterで使うURLに変換する / Convert the GitHub URL to the URL used by NNPoster.
github_url = "" #@param {type:"string"}

import re

def convert_github_url_to_raw(url):
    pattern = r"https://github.com/([^/]+)/([^/]+)/blob/([^/]+)/(.+)"
    replacement = r"https://raw.githubusercontent.com/\1/\2/\3/\4"
    raw_url = re.sub(pattern, replacement, url)
    return raw_url

raw_url = convert_github_url_to_raw(github_url)
print("GitHub URL":"+ raw_url)
